In [8]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [9]:
mlflow.dspy.autolog()

In [1]:
import dspy

gpt4o_mini = dspy.LM('openai/gpt-4o-mini', max_tokens=2000)
gpt4o = dspy.LM('openai/gpt-4o', max_tokens=2000)
dspy.configure(lm=gpt4o_mini)  # we'll use gpt-4o-mini as the default LM, unless otherwise specified

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dspy.datasets import MATH

dataset = MATH(subset='algebra')
print(len(dataset.train), len(dataset.dev))

Generating test split: 100%|██████████| 1187/1187 [00:00<00:00, 414024.02 examples/s]

350 350


In [3]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)

Question: The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.
Answer: 221


In [10]:
module = dspy.ChainOfThought("question -> answer")
module(question=example.question)

Prediction(
    reasoning="Cal O'Ree's weight loss each week is $1\\%$ of his weight at the end of the previous week. This means that at the end of each week, he will weigh $99\\%$ of his weight from the previous week. \n\nWe can express this mathematically. If his initial weight is $W_0 = 244$ pounds, then his weight at the end of week $n$ can be calculated using the formula:\n\n\\[\nW_n = W_{n-1} \\times 0.99\n\\]\n\nThis can be simplified to:\n\n\\[\nW_n = W_0 \\times (0.99)^n\n\\]\n\nFor $n = 10$ weeks, we have:\n\n\\[\nW_{10} = 244 \\times (0.99)^{10}\n\\]\n\nNow we calculate $(0.99)^{10}$:\n\n\\[\n(0.99)^{10} \\approx 0.904382\n\\]\n\nNow we can calculate Cal O'Ree's weight after 10 weeks:\n\n\\[\nW_{10} \\approx 244 \\times 0.904382 \\approx 220.5\n\\]\n\nRounding to the nearest whole number, Cal O'Ree can expect to weigh approximately $221$ pounds at the end of the ten weeks.",
    answer='221'
)

Trace(request_id=fe47f1f8059e4ee7888525bf0322b6a2)

In [7]:
dspy.inspect_history(n=1)





[2025-03-13T12:44:05.583790]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## a

In [5]:
THREADS = 24
kwargs = dict(num_threads=THREADS, display_progress=True, display_table=5)
evaluate = dspy.Evaluate(devset=dataset.dev, metric=dataset.metric, **kwargs)

evaluate(module)

Average Metric: 24.00 / 27 (88.9%):   7%|▋         | 26/350 [00:09<02:14,  2.41it/s]

2025/03/13 12:43:52 WARNING dspy.utils.parallelizer: SIGINT received. Cancelling.


KeyboardInterrupt: 